In [2]:
import pandas as pd
import os

# Read all the CSV files
print("Reading CSV files...")
product_label = pd.read_csv('product_label.csv')
vocab_meddra_adverse_effect = pd.read_csv('vocab_meddra_adverse_effect.csv')
vocab_rxnorm_ingredient = pd.read_csv('vocab_rxnorm_ingredient.csv')
vocab_rxnorm_product = pd.read_csv('vocab_rxnorm_product.csv')
product_adverse_effect = pd.read_csv('product_adverse_effect.csv')
product_to_rxnorm = pd.read_csv('product_to_rxnorm.csv')
vocab_rxnorm_ingredient_to_product = pd.read_csv('vocab_rxnorm_ingredient_to_product.csv')

Reading CSV files...


C:\Users\zakar\AppData\Local\Temp\ipykernel_30952\4211879872.py:10: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  product_adverse_effect = pd.read_csv('product_adverse_effect.csv')


In [3]:
print("Data loaded successfully!")
print("\nTable shapes:")
print(f"product_label: {product_label.shape}")
print(f"vocab_meddra_adverse_effect: {vocab_meddra_adverse_effect.shape}")
print(f"vocab_rxnorm_ingredient: {vocab_rxnorm_ingredient.shape}")
print(f"vocab_rxnorm_product: {vocab_rxnorm_product.shape}")
print(f"product_adverse_effect: {product_adverse_effect.shape}")
print(f"product_to_rxnorm: {product_to_rxnorm.shape}")
print(f"vocab_rxnorm_ingredient_to_product: {vocab_rxnorm_ingredient_to_product.shape}")

Data loaded successfully!

Table shapes:
product_label: (59515, 5)
vocab_meddra_adverse_effect: (7177, 3)
vocab_rxnorm_ingredient: (2562, 3)
vocab_rxnorm_product: (18594, 3)
product_adverse_effect: (28126418, 7)
product_to_rxnorm: (94608, 2)
vocab_rxnorm_ingredient_to_product: (11590, 2)


In [4]:
# Step 1: Join product_label with product_to_rxnorm to get label_id to rxnorm_product_id mapping
print("\nStep 1: Joining product_label with product_to_rxnorm...")
label_to_product = pd.merge(
    product_label[['label_id']],
    product_to_rxnorm,
    on='label_id',
    how='inner'
)


Step 1: Joining product_label with product_to_rxnorm...


In [5]:
# Step 2: Join with vocab_rxnorm_ingredient_to_product to get ingredient_ids
print("Step 2: Adding ingredient information...")
label_to_ingredient = pd.merge(
    label_to_product,
    vocab_rxnorm_ingredient_to_product,
    left_on='rxnorm_product_id',
    right_on='product_id',
    how='inner'
)

Step 2: Adding ingredient information...


In [6]:
# Step 3: Join with product_adverse_effect to get meddra_ids
print("Step 3: Adding adverse effect information...")
ingredient_to_meddra = pd.merge(
    label_to_ingredient,
    product_adverse_effect[['product_label_id', 'effect_meddra_id']],
    left_on='label_id',
    right_on='product_label_id',
    how='inner'
)

Step 3: Adding adverse effect information...


In [7]:
# Step 4: Join with vocab_meddra_adverse_effect to get meddra_names
print("Step 4: Adding meddra names...")
final_result = pd.merge(
    ingredient_to_meddra,
    vocab_meddra_adverse_effect[['meddra_id', 'meddra_name']],
    left_on='effect_meddra_id',
    right_on='meddra_id',
    how='inner'
)

Step 4: Adding meddra names...


In [8]:
# Step 5: Select and rename the final columns
print("Step 5: Creating final table...")
rxnorm_ingredient_meddra = final_result[[
    'ingredient_id', 'meddra_id', 'meddra_name'
]].drop_duplicates().reset_index(drop=True)

# Rename columns for clarity
rxnorm_ingredient_meddra = rxnorm_ingredient_meddra.rename(
    columns={'ingredient_id': 'rxnorm_ingredient_id'}
)


Step 5: Creating final table...


In [9]:

print(f"\nFinal table shape: {rxnorm_ingredient_meddra.shape}")
print(f"Unique rxnorm_ingredient_ids: {rxnorm_ingredient_meddra['rxnorm_ingredient_id'].nunique()}")
print(f"Unique meddra_ids: {rxnorm_ingredient_meddra['meddra_id'].nunique()}")


Final table shape: (122353, 3)
Unique rxnorm_ingredient_ids: 2547
Unique meddra_ids: 5909


In [10]:

# Save as Parquet file
print("\nSaving as Parquet file...")
rxnorm_ingredient_meddra.to_parquet('rxnorm_ingredient_to_meddra.parquet', index=False)


Saving as Parquet file...
